In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random
from tqdm.notebook import tqdm


from modules.dataset import IntelImageClassificationDataset
from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
from modules.trainer import Trainer

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

/home/ESPL_001/dez5881/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


cuda


In [3]:
choice = 1 # 1,2,3

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# 80% train, 20% validation for training Optuna
train_size = int(0.8 * len(dataset.train_dataset))
val_size = len(dataset.train_dataset) - train_size
train_subset, val_subset = random_split(dataset.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(1))

def build_model():

  # SqueezeNet 1.1
  if choice == 1:
      model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
      num_features = model.classifier[1].in_channels
      kernel_size = model.classifier[1].kernel_size
      model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)


  # MobileNetV2
  elif choice == 2:
      model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
      num_features = model.classifier[1].in_features
      model.classifier[1] = nn.Linear(num_features, 6)

  # MobileNetV3 Small
  elif choice == 3:
      model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      model.classifier[3] = nn.Linear(num_features, 6)

  # MobileNetV3 Large
  elif choice == 4:
      model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      model.classifier[3] = nn.Linear(num_features, 6)

  # VisionTransformer Base 16
  elif choice == 5:
      model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)
      num_features = model.heads[0].in_features
      model.heads[0] = nn.Linear(num_features, 6)

  return model


model = build_model()
trainDataloader = DataLoader(dataset.train_dataset, batch_size=320, num_workers=4, shuffle=True)
testDataloader = DataLoader(dataset.test_dataset, batch_size=len(dataset.test_dataset), shuffle=False)
trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)

In [4]:
print(len(dataset.train_dataset))

14034


In [5]:
epochs = 10
lr=8.841926348917726e-05
model = model.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr)
loss = nn.CrossEntropyLoss()
scaler = torch.amp.GradScaler()

prof = torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=2, warmup=3, active=10, repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/trainer_profile'),
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
    with_flops=True,
    with_modules=True
    )
prof.start()

In [6]:
fp16 = False
for epoch in tqdm(range(epochs)):
    model.train()
    if fp16:
        for i, (data, target) in enumerate(trainDataloader):
            optimizer.zero_grad()
            x = data.to(DEVICE)
            y = target.to(DEVICE)

            with torch.amp.autocast(device_type="cuda"):
                x = model(x)

            l = loss(x, y)

            scaler.scale(l).backward()
            scaler.step(optimizer)
            scaler.update()

            prof.step()
    else:
        for i, (data, target) in enumerate(trainDataloader):
            optimizer.zero_grad()
            x = data.to(DEVICE)
            y = target.to(DEVICE)

            x = model(x)
            l = loss(x, y)

            l.backward()
            optimizer.step()

            prof.step()

prof.stop()

  0%|          | 0/10 [00:00<?, ?it/s]

# model.load_state_dict(torch.load(f"checkpoints/.pt"))
trainer.train(dataloader, epochs=epochs, silent=False)

In [6]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

0.904121994972229

In [7]:
# torch.save(model.state_dict(), f"checkpoints/{model.__class__.__name__}.pt")

## Initial Results for Model Selection

| model | accuracy | size |
| --- | --- | --- |
| ResNet18 | 0.87 | 44.7 MB |
| ResNet34 | 0.88 | 83.3 MB |
| MobileNet V2 | 0.91 | 13.6 MB |
| MobileNet V3 small | 0.90 | 9.8 MB |
| VGG19 | 0.83 | 548.1 MB |
| SqueezeNet 1.0 | 0.89 | 4.8 MB |
| DenseNet | 0.90 | 30.8 MB |
| EfficientNet B0 | 0.92 | 20.5 MB |
| ViT-b/16 | 0.73 | 330.3 MB |